<a href="https://colab.research.google.com/github/rahitya-123/SnowPark-Introduction/blob/main/snowpark_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install snowflake-snowpark-python
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
from datetime import date

In [ ]:
!pip install --upgrade pip

In [ ]:
from google.colab import userdata


In [ ]:
def snowpark_sesion_create():
  account=userdata.get('account')
  user=userdata.get('user')
  password=userdata.get('password')
  role=userdata.get('role')


  connection_params={
      "account":account,
      "user": user,
      "password": password,
      "role": role
  }
  session = Session.builder.configs(connection_params).create()
  return session

In [ ]:
demo_session = snowpark_sesion_create()

In [ ]:
df=demo_session.sql('select * from snowflake_sample_data.tpch_sf10.lineitem limit 100')

In [ ]:
df.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"L_ORDERKEY"  |"L_PARTKEY"  |"L_SUPPKEY"  |"L_LINENUMBER"  |"L_QUANTITY"  |"L_EXTENDEDPRICE"  |"L_DISCOUNT"  |"L_TAX"  |"L_RETURNFLAG"  |"L_LINESTATUS"  |"L_SHIPDATE"  |"L_COMMITDATE"  |"L_RECEIPTDATE"  |"L_SHIPINSTRUCT"   |"L_SHIPMODE"  |"L_COMMENT"                               |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|6600001       |1267154      |67155        |1               |23.00         |25785.07           |0.08          |0.06     |N               |O         

In [ ]:
df.collect()

In [ ]:
#point the session to snowflake_sample_data database
demo_session.use_database('snowflake_sample_data')
#point session to the tpch_sf10 schema
demo_session.use_schema('tpch_sf10')

In [ ]:
#set the table to lineitem
df=demo_session.table('lineitem')

#filter L_SHIPDATE to just dates in 1997
df=df.select('L_SHIPDATE',
             'L_EXTENDEDPRICE',
             'L_SHIPMODE',
             'L_QUANTITY',
             'L_DISCOUNT'
             ).filter(F.col('L_SHIPDATE').between(date(1997,1,1),date(1997,12,31)))

In [ ]:
#show the result
df.show()

---------------------------------------------------------------------------------
|"L_SHIPDATE"  |"L_EXTENDEDPRICE"  |"L_SHIPMODE"  |"L_QUANTITY"  |"L_DISCOUNT"  |
---------------------------------------------------------------------------------
|1997-04-30    |57804.60           |SHIP          |42.00         |0.08          |
|1997-06-21    |51600.64           |SHIP          |32.00         |0.06          |
|1997-05-31    |70673.82           |MAIL          |42.00         |0.10          |
|1997-07-14    |50614.06           |FOB           |49.00         |0.08          |
|1997-06-26    |77449.89           |MAIL          |49.00         |0.01          |
|1997-05-29    |58191.30           |MAIL          |45.00         |0.08          |
|1997-05-23    |61273.86           |AIR           |38.00         |0.06          |
|1997-08-06    |1744.63            |REG AIR       |1.00          |0.04          |
|1997-07-05    |65080.07           |MAIL          |43.00         |0.01          |
|1997-04-06    |

In [ ]:
#create a TOTAL_REVENUE column which is (L_EXTENDEDPRICE-(L_EXTENDEDPRICE*L_DISCOUNT))*L_QUNATITY
df=df.with_column('REVENUE',
                   (F.col('L_EXTENDEDPRICE')-(F.col('L_EXTENDEDPRICE')*F.col('L_DISCOUNT'))) * F.col('L_QUANTITY'))

In [ ]:
#show the result
df.show()

--------------------------------------------------------------------------------------------------
|"L_SHIPDATE"  |"L_EXTENDEDPRICE"  |"L_SHIPMODE"  |"L_QUANTITY"  |"L_DISCOUNT"  |"REVENUE"       |
--------------------------------------------------------------------------------------------------
|1997-04-30    |57804.60           |SHIP          |42.00         |0.08          |2233569.744000  |
|1997-06-21    |51600.64           |SHIP          |32.00         |0.06          |1552147.251200  |
|1997-05-31    |70673.82           |MAIL          |42.00         |0.10          |2671470.396000  |
|1997-07-14    |50614.06           |FOB           |49.00         |0.08          |2281681.824800  |
|1997-06-26    |77449.89           |MAIL          |49.00         |0.01          |3757094.163900  |
|1997-05-29    |58191.30           |MAIL          |45.00         |0.08          |2409119.820000  |
|1997-05-23    |61273.86           |AIR           |38.00         |0.06          |2188702.279200  |
|1997-08-0

In [ ]:
#create a new Df called total1997 which groups by L_SHIPMODE and gives us the total revenue by ship mode
total1997=df.group_by('L_SHIPMODE').agg(F.sum('REVENUE').alias('TOTAL_REVENUE'))

In [ ]:
#show the result
total1997.show()

---------------------------------------
|"L_SHIPMODE"  |"TOTAL_REVENUE"       |
---------------------------------------
|AIR           |1588489012314.814700  |
|TRUCK         |1590733688302.036700  |
|SHIP          |1587705642771.441000  |
|MAIL          |1589584697330.409300  |
|RAIL          |1593265749227.381000  |
|FOB           |1588274810649.676300  |
|REG AIR       |1588413005265.937100  |
---------------------------------------



In [ ]:
df2=demo_session.table('lineitem')

#filter L_SHIPDATE to just dates in 1998
df2=df2.select('L_SHIPDATE',
             'L_EXTENDEDPRICE',
             'L_SHIPMODE',
             'L_QUANTITY',
             'L_DISCOUNT'
             ).filter(F.col('L_SHIPDATE').between(date(1998,1,1),date(1998,12,31)))

df2=df2.with_column('REVENUE',
                   (F.col('L_EXTENDEDPRICE')-(F.col('L_EXTENDEDPRICE')*F.col('L_DISCOUNT'))) * F.col('L_QUANTITY'))


#create a new Df called total1998 which groups by L_SHIPMODE and gives us the total revenue by ship mode
total1998=df2.group_by('L_SHIPMODE').agg(F.sum('REVENUE').alias('TOTAL_REVENUE'))

In [ ]:
#show the result
total1998=total1998.with_column_renamed(F.col('TOTAL_REVENUE'),'TOTAL_REVENUE_1998')
total1998.show()

---------------------------------------
|"L_SHIPMODE"  |"TOTAL_REVENUE_1998"  |
---------------------------------------
|TRUCK         |1197327971050.476000  |
|FOB           |1192940262362.108000  |
|SHIP          |1198711220773.695500  |
|RAIL          |1196094347242.170000  |
|MAIL          |1198424237452.120300  |
|AIR           |1198501334545.354000  |
|REG AIR       |1197951763139.771600  |
---------------------------------------



In [ ]:
#Rename the column TOTAL_REVENUE in total1997 to TOTAL_REVENUE_1997
total1997=total1997.with_column_renamed(F.col('TOTAL_REVENUE'),'TOTAL_REVENUE_1997')

In [ ]:
#join the two dataframes
total1997.join(total1998,total1997['L_SHIPMODE']==total1998['L_SHIPMODE']).show()

-----------------------------------------------------------------------------------------------
|"l_3pyw_L_SHIPMODE"  |"l_3pyw_TOTAL_REVENUE"  |"r_ugls_L_SHIPMODE"  |"r_ugls_TOTAL_REVENUE"  |
-----------------------------------------------------------------------------------------------
|SHIP                 |1587705642771.441000    |SHIP                 |1198711220773.695500    |
|MAIL                 |1589584697330.409300    |MAIL                 |1198424237452.120300    |
|RAIL                 |1593265749227.381000    |RAIL                 |1196094347242.170000    |
|TRUCK                |1590733688302.036700    |TRUCK                |1197327971050.476000    |
|AIR                  |1588489012314.814700    |AIR                  |1198501334545.354000    |
|FOB                  |1588274810649.676300    |FOB                  |1192940262362.108000    |
|REG AIR              |1588413005265.937100    |REG AIR              |1197951763139.771600    |
----------------------------------------

In [ ]:
#join the two dataframes
joined_dfs=total1997.join(total1998,total1997['L_SHIPMODE']==total1998['L_SHIPMODE']).select(
    total1997.col('L_SHIPMODE').alias('L_SHIPMODE'),
    total1997.col('TOTAL_REVENUE_1997'),
    total1998.col('TOTAL_REVENUE_1998')
    )

In [ ]:
#create a coulmn that puts the two totals in an array
#create a column that adds a comment specifying who created the logic
#create a column that says that the trust level of REG AIR is 40% and , for all other ship modes is 100%
joined_dfs=joined_dfs.with_column(
    'TOTALS',
    F.array_construct(F.col('TOTAL_REVENUE_1997'),F.col('TOTAL_REVENUE_1998'))
).with_column(
    'COMMENT',
    F.concat(F.lit("Logic was created by "),F.current_user())
).with_column(
    'TRUST_LEVEL',
    F.when(F.col('L_SHIPMODE')=="REG AIR",0.4).
    otherwise(1)
)

In [ ]:
#Drop columns TOTAL_REVENUE_1997 and TOTAL_REVENUE_1998
joined_dfs=joined_dfs.drop('TOTAL_REVENUE_1997','TOTAL_REVENUE_1998')

In [ ]:
#show the result
joined_dfs.show()

------------------------------------------------------------------------------------------
|"L_SHIPMODE"  |"TOTALS"               |"COMMENT"                        |"TRUST_LEVEL"  |
------------------------------------------------------------------------------------------
|AIR           |[                      |Logic was created by RAHITYA137  |1.0            |
|              |  1588489012314.8147,  |                                 |               |
|              |  1198501334545.354    |                                 |               |
|              |]                      |                                 |               |
|SHIP          |[                      |Logic was created by RAHITYA137  |1.0            |
|              |  1587705642771.441,   |                                 |               |
|              |  1198711220773.6955   |                                 |               |
|              |]                      |                                 |               |

In [ ]:
#close the session
demo_session.close()